In [1]:
%pip install git+https://github.com/tinygrad/tinygrad.git

  Cloning https://github.com/tinygrad/tinygrad.git to /tmp/pip-req-build-ju0y_nmx
  Running command git clone --filter=blob:none --quiet https://github.com/tinygrad/tinygrad.git /tmp/pip-req-build-ju0y_nmx
  Resolved https://github.com/tinygrad/tinygrad.git to commit 44558a37f76ab75964427a9d192e69cddf148312
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for tinygrad: filename=tinygrad-0.11.0-py3-none-any.whl size=1766961 sha256=4746851d518288665ff409816740ee3e19f40ab86298ab8da8fdc0d0af549195
  Stored in directory: /tmp/pip-ephem-wheel-cache-dbt8sr1k/wheels/8f/3f/85/ab4544b53ccc541baaff911f7bb60c606a5a9f1b71016b77b4
Successfully built tinygrad
Note: you may need to restart the kernel to use updated packages.


In [2]:
from tinygrad import Device
print(Device.DEFAULT)

CPU


In [13]:
from tinygrad import Tensor, nn

In [14]:
class Model:
    def __init__(self):
        self.l1 = nn.Conv2d(1, 32, kernel_size=(3, 3))
        self.l2 = nn.Conv2d(32, 64, kernel_size=(3, 3))
        self.l3 = nn.Linear(1600, 10)

    def __call__(self, x : tensor) -> tensor:
        x = self.l1(x).relu().max_pool2d((2, 2))
        x = self.l2(x).relu().max_pool2d((2, 2))
        return self.l3(x.flatten(1).dropout(0.5))


stateful layers ??? 

In [ ]:
from tinygrad.nn.datatensor.sets import mnist

In [16]:
x_train, y_train, x_test, y_test = mnist()

In [17]:
model = Model()
acc = (model(x_test).argmax(axis=1) == y_test).mean()
print(acc.item())

0.1078999936580658


In [19]:
optim = nn.optim.Adam(nn.state.get_parameters(model))
batch_size = 128
def step():
    Tensor.training = True
    samples = Tensor.randint(batch_size, high=x_train.shape[0])
    x, y = x_train[samples], y_train[samples]
    optim.zero_grad()
    loss = model(x).sparse_categorical_crossentropy(y).backward()
    optim.step()
    return loss

In [20]:
from tinygrad import GlobalCounters, Context
GlobalCounters.reset()
with Context(DEBUG = 2): 
    step()

DISK:/home/project-un/.cache/tinygrad/downloads/e8e6ed0857e9bb000985bfea8283b0b3.gunzip: using Compiler
opened device DISK:/home/project-un/.cache/tinygrad/downloads/e8e6ed0857e9bb000985bfea8283b0b3.gunzip from pid:79242
DISK:/home/project-un/.cache/tinygrad/downloads/684fe191a76d9107981692e6f0d3842f.gunzip: using Compiler
opened device DISK:/home/project-un/.cache/tinygrad/downloads/684fe191a76d9107981692e6f0d3842f.gunzip from pid:79242
scheduled 75 kernels in 174.39 ms
*** CPU        1 En13                                         arg  1 mem  0.01 GB tm     50.65us/     0.05ms (     0.00 GFLOPS    0.0|0.0     GB/s) ['conv2d']
*** CPU        2 E_72_4n2                                     arg  1 mem  0.01 GB tm     20.96us/     0.07ms (     0.00 GFLOPS    0.1|0.1     GB/s) ['conv2d']
*** CPU        3 E_16_4n2                                     arg  1 mem  0.01 GB tm     49.93us/     0.12ms (     0.00 GFLOPS    0.0|0.0     GB/s) ['conv2d']
*** CPU        4 En14                          

In [21]:
from tinygrad import TinyJit

In [22]:
jitstep = TinyJit(step)

In [23]:
import timeit 
timeit.repeat(jitstep, repeat=5, number=1)

[1.2525726229978318,
 0.1375934360003157,
 0.004452660999959335,
 0.29407747000004747,
 0.07500287200309685]

In [24]:
for step in range(7000):
    loss = jitstep()
    if step % 100 == 0:
        Tensor.training = False
        acc = (model(x_test).argmax(axis=1) == y_test).mean().item()
        print(f"step {step:4d}, loss {loss.item():.2f}, acc {acc*100.:.2f}%")

step    0, loss 5.79, acc 62.89%
step  100, loss 0.35, acc 95.06%
step  200, loss 0.12, acc 96.93%
step  300, loss 0.16, acc 97.00%
step  400, loss 0.18, acc 97.42%
step  500, loss 0.13, acc 97.58%
step  600, loss 0.14, acc 97.84%
step  700, loss 0.07, acc 97.97%
step  800, loss 0.15, acc 97.68%
step  900, loss 0.22, acc 98.13%
step 1000, loss 0.16, acc 98.10%
step 1100, loss 0.25, acc 98.18%
step 1200, loss 0.09, acc 98.17%
step 1300, loss 0.06, acc 98.30%
step 1400, loss 0.13, acc 98.07%
step 1500, loss 0.05, acc 98.41%
step 1600, loss 0.09, acc 98.31%
step 1700, loss 0.10, acc 98.37%
step 1800, loss 0.11, acc 98.43%
step 1900, loss 0.06, acc 98.19%
step 2000, loss 0.05, acc 98.39%
step 2100, loss 0.06, acc 98.51%
step 2200, loss 0.11, acc 98.48%
step 2300, loss 0.07, acc 98.71%
step 2400, loss 0.03, acc 98.53%
step 2500, loss 0.07, acc 98.57%
step 2600, loss 0.07, acc 98.73%
step 2700, loss 0.10, acc 98.72%
step 2800, loss 0.09, acc 98.74%
step 2900, loss 0.02, acc 98.72%
step 3000,

In [25]:
acc = (model(x_test).argmax(axis=1) == y_test).mean()
print(acc.item())

0.9894999861717224
